#Simple Option Trading Strategies
Create Backtesting environment for option based strategies



#### Libraries

In [ ]:
!pip install alpaca-py
!pip install mibian

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mibian: filename=mibian-0.1.3-py3-none-any.whl size=4025 sha256=1d92114b82ad3bdd9f0d3c66e00e196490408f9bc4af4bb3744b35caa5deb425
  Stored in directory: /root/.cache/pip/wheels/2c/4f/a7/be034e17cc306b0850f5f1a5b4541281b49475c58620a7ff40
Successfully built mibian


In [ ]:
import numpy as np
import pandas as pd
#Alpaca
import alpaca
from alpaca.trading.client import *
from alpaca.data.historical.stock import *
from alpaca.data.historical.crypto import *
from alpaca.data.historical.option import *
from alpaca.trading.requests import *
from alpaca.data.timeframe import *

import time
from datetime import datetime, timedelta, date
from zoneinfo import ZoneInfo

import mibian

In [ ]:
# Alpaca API credentials
api_key = "PKRYJ6L3JXDKEDHN0BCM"
secret_key = "dEDO2DAIUhLBBd3fix8Hmddp5J6gXB6VPQo9l9va"
base_url = 'https://paper-api.alpaca.markets'  # Use the appropriate URL for your account type

base_url ='https://paper-api.alpaca.markets'  # Use the appropriate URL for your account type

trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=True, url_override=base_url)
trade_client.get_account()

{   'account_blocked': False,
    'account_number': 'PA3B1P8C1FGR',
    'accrued_fees': '0',
    'buying_power': '2000000',
    'cash': '1000000',
    'created_at': datetime.datetime(2024, 5, 14, 11, 19, 25, 731640, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '1000000',
    'id': UUID('b802565c-7d6e-4e1f-bc6d-40f92a261bb1'),
    'initial_margin': '0',
    'last_equity': '1000000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '1000000',
    'options_approved_level': 2,
    'options_buying_power': '1000000',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'pending_transfer_out': None,
    'portfolio_value': '1000000',
    'regt_buying_power': '2000000',
    'short_market_value': '0',
    'shorting_en

### Functions

In [ ]:
import numpy as np
from scipy.stats import norm

def calculate_call_option_greeks(S, K, T, r, sigma):
    """
    Calculate the Greeks for a European call option using the Black-Scholes model.

    Parameters:
        S (float): Current stock price
        K (float): Strike price of the option
        T (float): Time to expiration in years
        r (float): Risk-free interest rate
        sigma (float): Volatility of the underlying asset
    """
    # Calculate d1 and d2 components of the Black-Scholes model
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = (S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))

    # Calculate Greeks for the call option
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = - (S * norm.pdf(d1) * sigma) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    vega = S * norm.pdf(d1) * np.sqrt(T)
    rho = K * T * np.exp(-r * T) * norm.cdf(d2)
    # Calculate Vanna and Vomma
    vanna = -norm.pdf(d1) * d2 / sigma
    vomma = S * norm.pdf(d1) * np.sqrt(T) * (d1 * d2) / sigma

    # Return the results as a dictionary
    return {
        "Delta": delta,
        "Gamma": gamma,
        "Theta": theta,
        "Vega": vega,
        "Rho": rho,
        "Vanna": vanna,
        "Vomma": vomma
    }

In [ ]:
def calculate_implied_volatility(S, K, T, r, market_price):
    """
    Calculate implied volatility using the Brent's method to find the root.

    Parameters:
        S (float): Current stock price
        K (float): Strike price of the option
        T (float): Time to expiration in years
        r (float): Risk-free interest rate
        market_price (float): Current market price of the call option

    Returns:
        float: Implied volatility
    """
    # Function to find the root
    def objective(sigma):
        return black_scholes_call_price(S, K, T, r, sigma) - market_price

    # Initial guess and bounds for sigma
    initial_guess = 0.2
    lower_bound = 0.001
    upper_bound = 2.0

    # Brent's method to solve for sigma
    implied_vol = brentq(objective, lower_bound, upper_bound)
    return implied_vol

### Data

In [ ]:
symbol = 'SPY'
# get list of assets which are options enabled
req = GetAssetsRequest(
  attributes = "options_enabled")
assets = trade_client.get_all_assets(req)
assets = pd.DataFrame.from_dict(list(map(dict,assets)))
assets[assets.symbol == symbol]

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement,attributes
4133,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,AssetClass.US_EQUITY,AssetExchange.ARCA,SPY,SPDR S&P 500 ETF Trust,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"


In [ ]:

# setup stock historical data client
shd_client = StockHistoricalDataClient(api_key, secret_key, url_override = None)
# get historical bars by symbol

req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe= TimeFrame(amount = 1, unit = TimeFrameUnit.Day), # specify timeframe
    start = '2024-01-01',                                         # specify start datetime, default=the beginning of the current day.
    #limit = 1000                                               # specify limit
)


ohlc_data = shd_client.get_stock_bars(req).df
ohlc_data = ohlc_data.loc[symbol]
ohlc_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2024-01-02 05:00:00+00:00,472.16,473.67,470.490,472.65,123007793.0,657585.0,472.509570
2024-01-03 05:00:00+00:00,470.43,471.19,468.170,468.79,103595966.0,656572.0,469.898320
2024-01-04 05:00:00+00:00,468.30,470.96,467.050,467.28,84232169.0,536471.0,468.554179
2024-01-05 05:00:00+00:00,467.49,470.44,466.430,467.92,86091270.0,561741.0,468.077782
2024-01-08 05:00:00+00:00,468.43,474.75,468.300,474.60,74879074.0,523806.0,471.967417
...,...,...,...,...,...,...,...
2024-05-24 04:00:00+00:00,527.85,530.27,526.881,529.44,41331076.0,328934.0,528.914223
2024-05-28 04:00:00+00:00,530.27,530.51,527.110,529.81,36269602.0,343542.0,529.320262
2024-05-29 04:00:00+00:00,525.68,527.31,525.370,526.10,45190323.0,359207.0,526.604624


First we calculate the necessary metrics for options

In [ ]:
req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe= TimeFrame(amount = 1, unit = TimeFrameUnit.Day), # specify timeframe
    start = '2023-01-01',                                         # specify start datetime, default=the beginning of the current day.
    #limit = 1000                                               # specify limit
    end_date = '2023-12-31'
)


past_data = shd_client.get_stock_bars(req).df
past_data = past_data.loc[symbol]
past_data['log_returns'] = np.log(past_data.close).diff()
hv = past_data['log_returns'].std()*np.sqrt(252)*100
hv

12.666489663294131

In [ ]:
symbol = 'SPY'
underlying_symbol = [symbol]
from alpaca.trading.enums import *
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now - timedelta(days = 30)
day120 = now + timedelta(days = 120)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbol,                     # specify underlying symbols
    expiration_date =None,                                      # specify expiration date (specified date + 1 day range)
    #expiration_date_gte = ,                           # we can pass date object
    #expiration_date_lte = day120.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = None,                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 1000,                                                 # specify limit
    page_token = None,                                           # specify page
    open_interest = 1
    )
res = trade_client.get_option_contracts(req)
options_df = pd.DataFrame(list(map(dict,res.option_contracts)))
#options_df = options_df[options_df.open_interest.astype(float) >= 10]
options_df

,id,symbol,name,status,tradable,expiration_date,root_symbol,underlying_symbol,underlying_asset_id,type,style,strike_price,size,open_interest,open_interest_date,close_price,close_price_date
0,286668c7-4b9b-4f67-9856-ae87e853f480,SPY240603C00451000,SPY Jun 03 2024 451 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,451.0,100,1,2024-05-31,71.17,2024-05-31
1,5c16d0d2-cf58-47c9-8d9c-821c963330df,SPY240603C00452000,SPY Jun 03 2024 452 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,452.0,100,None,None,70.17,2024-05-31
2,a6793283-b85a-49c3-bf4d-4af8b5b9b09c,SPY240603C00453000,SPY Jun 03 2024 453 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,453.0,100,None,None,None,None
3,10a84589-1a2c-4999-9b87-0ac3229190d4,SPY240603C00454000,SPY Jun 03 2024 454 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,454.0,100,64,2024-05-31,69.99,2024-05-30
4,ee8396ad-6918-4f6b-a902-a4aad138d0fc,SPY240603C00455000,SPY Jun 03 2024 455 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,455.0,100,1,2024-05-31,66,2024-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,73be8b4b-a287-485b-9f6a-a3dcd74e921f,SPY240607C00528000,SPY Jun 07 2024 528 Call,AssetStatus.ACTIVE,True,2024-06-07,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,528.0,100,3086,2024-05-31,2.94,2024-05-31
996,90d0298f-e3e8-4745-a49d-6d8f8ebd92ff,SPY240607C00529000,SPY Jun 07 2024 529 Call,AssetStatus.ACTIVE,True,2024-06-07,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,529.0,100,3246,2024-05-31,2.39,2024-05-31
997,713393a5-8b5b-42bd-8ce9-e242cce87951,SPY240607C00530000,SPY Jun 07 2024 530 Call,AssetStatus.ACTIVE,True,2024-06-07,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,530.0,100,11473,2024-05-31,1.96,2024-05-31
998,a6c8c24e-d427-4396-86a4-c03a91249f3c,SPY240607C00531000,SPY Jun 07 2024 531 Call,AssetStatus.ACTIVE,True,2024-06-07,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,531.0,100,7688,2024-05-31,1.53,2024-05-31


In [ ]:
#Find at the monery option at 2024-01-01
at_price = ohlc_data.close[-1]
at_money_op = options_df[(options_df.strike_price >= at_price-2)&(options_df.strike_price <= at_price+5)
          &(options_df.type == ContractType.CALL)]
at_money_op

,id,symbol,name,status,tradable,expiration_date,root_symbol,underlying_symbol,underlying_asset_id,type,style,strike_price,size,open_interest,open_interest_date,close_price,close_price_date
75,c68d41ea-b757-47a2-bb61-f50d6b55239a,SPY240603C00526000,SPY Jun 03 2024 526 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,526.0,100,4027,2024-05-31,2.17,2024-05-31
76,54ea6fe4-dc87-4afd-9a26-cdd1fae39fc0,SPY240603C00527000,SPY Jun 03 2024 527 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,527.0,100,4056,2024-05-31,1.55,2024-05-31
77,2e3086e1-a478-4fb1-ac0e-8add636cb40c,SPY240603C00528000,SPY Jun 03 2024 528 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,528.0,100,3703,2024-05-31,0.99,2024-05-31
78,f2e042b4-0406-46f5-88c2-45ec13f63779,SPY240603C00529000,SPY Jun 03 2024 529 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,529.0,100,3189,2024-05-31,0.58,2024-05-31
79,151a139c-fa8f-432e-bce8-41f78c3e890b,SPY240603C00530000,SPY Jun 03 2024 530 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,530.0,100,7333,2024-05-31,0.28,2024-05-31
80,3a49c91c-ff45-49fd-9d82-5d1b11ecba1c,SPY240603C00531000,SPY Jun 03 2024 531 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,531.0,100,3141,2024-05-31,0.13,2024-05-31
81,a0e2de4a-7150-4b7f-a5a8-dbcf4782a0e2,SPY240603C00532000,SPY Jun 03 2024 532 Call,AssetStatus.ACTIVE,True,2024-06-03,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,532.0,100,2424,2024-05-31,0.05,2024-05-31
299,61c7c60d-d889-4854-89b1-1e64c05b3cf1,SPY240604C00526000,SPY Jun 04 2024 526 Call,AssetStatus.ACTIVE,True,2024-06-04,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,526.0,100,3024,2024-05-31,2.67,2024-05-31
300,c0258f9a-578f-4b91-9663-70282e66b654,SPY240604C00527000,SPY Jun 04 2024 527 Call,AssetStatus.ACTIVE,True,2024-06-04,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,527.0,100,2605,2024-05-31,2.06,2024-05-31
301,6092be6e-eef6-4936-a27f-021747abd349,SPY240604C00528000,SPY Jun 04 2024 528 Call,AssetStatus.ACTIVE,True,2024-06-04,SPY,SPY,b28f4066-5c6d-479b-a2af-85dc1a8f16fb,ContractType.CALL,ExerciseStyle.AMERICAN,528.0,100,2231,2024-05-31,1.5,2024-05-31


In [ ]:
t = 2
at_money_op_symbol = at_money_op.symbol.iloc[t]

st_price = at_money_op.strike_price.iloc[t]
exp_date = at_money_op.expiration_date.iloc[t]
at_money_op_symbol
# setup option historical data client
ohd_client = OptionHistoricalDataClient(api_key, secret_key, url_override = None)
# get options historical bars by symbol
req = OptionBarsRequest(
    symbol_or_symbols = at_money_op_symbol,
    timeframe = TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),   # specify timeframe
    start = '2024-01-01',
    #end_date='2024-05-01',                                                # specify end datetime, default=now
)
option_prices = ohd_client.get_option_bars(req).df#.loc[at_money_op_symbol]
option_prices

open   high   low  close  \
symbol             timestamp                                              
SPY240603C00528000 2024-05-28 13:00:00+00:00   4.06   4.20  3.59   3.62   
                   2024-05-28 14:00:00+00:00   3.77   4.05  3.70   4.05   
                   2024-05-28 15:00:00+00:00   3.80   3.93  3.59   3.93   
                   2024-05-28 16:00:00+00:00   3.85   4.05  3.76   3.78   
                   2024-05-28 17:00:00+00:00   3.45   3.74  3.15   3.28   
                   2024-05-28 18:00:00+00:00   3.21   3.44  2.45   2.89   
                   2024-05-28 19:00:00+00:00   2.85   3.78  2.79   3.78   
                   2024-05-28 20:00:00+00:00   4.06   4.06  3.58   3.58   
                   2024-05-29 13:00:00+00:00   1.64   1.85  1.55   1.79   
                   2024-05-29 14:00:00+00:00   1.77   2.24  1.73   2.16   
                   2024-05-29 15:00:00+00:00   2.15   2.15  1.79   1.96   
                   2024-05-29 16:00:00+00:00   1.87   2.32  1.82   2.18   
                   2024-05-29 17:00:00+00:00   2.14   2.22  1.79   2.13   
                   2024-05-29 18:00:00+00:00   2.10   2.18  1.74   1.77   
                   2024-05-29 19:00:00+00:00   1.79   2.15  1.58   1.68   
                   2024-05-29 20:00:00+00:00   1.76   1.76  1.50   1.50   
                   2024-05-30 13:00:00+00:00   0.95   1.13  0.86   0.96   
                   2024-05-30 14:00:00+00:00   0.98   1.02  0.61   0.68   
                   2024-05-30 15:00:00+00:00   0.67   0.85  0.60   0.72   
                   2024-05-30 16:00:00+00:00   0.72   0.91  0.72   0.90   
                   2024-05-30 17:00:00+00:00   0.98   1.05  0.85   0.92   
                   2024-05-30 18:00:00+00:00   0.89   0.90  0.67   0.74   
                   2024-05-30 19:00:00+00:00   0.74   0.74  0.39   0.52   
                   2024-05-30 20:00:00+00:00   0.42   0.44  0.34   0.34   
                   2024-05-31 13:00:00+00:00   0.38   0.50  0.24   0.27   
                   2024-05-31 14:00:00+00:00   0.27   0.27  0.08   0.09   
                   2024-05-31 15:00:00+00:00   0.09   0.09  0.04   0.05   
                   2024-05-31 16:00:00+00:00   0.05   0.05  0.03   0.03   
                   2024-05-31 17:00:00+00:00   0.05   0.06  0.04   0.05   
                   2024-05-31 18:00:00+00:00   0.04   0.06  0.03   0.03   
                   2024-05-31 19:00:00+00:00   0.04   0.84  0.03   0.66   
                   2024-05-31 20:00:00+00:00   0.89   1.50  0.82   0.99   
                   2024-06-01 13:00:00+00:00  10.00  10.00  9.90   9.90   
                   2024-06-01 14:00:00+00:00   9.90   9.90  9.90   9.90   

                                               volume  trade_count      vwap  
symbol             timestamp                                                  
SPY240603C00528000 2024-05-28 13:00:00+00:00    108.0         28.0  3.837593  
                   2024-05-28 14:00:00+00:00     27.0         12.0  3.829259  
                   2024-05-28 15:00:00+00:00     58.0         17.0  3.736379  
                   2024-05-28 16:00:00+00:00     34.0          7.0  3.899118  
                   2024-05-28 17:00:00+00:00    163.0         24.0  3.377976  
                   2024-05-28 18:00:00+00:00    430.0         98.0  2.728000  
                   2024-05-28 19:00:00+00:00    345.0         57.0  3.071797  
                   2024-05-28 20:00:00+00:00     56.0         11.0  3.686250  
                   2024-05-29 13:00:00+00:00    411.0        103.0  1.729659  
                   2024-05-29 14:00:00+00:00    732.0        225.0  1.964467  
                   2024-05-29 15:00:00+00:00   1075.0         64.0  1.936558  
                   2024-05-29 16:00:00+00:00    376.0        105.0  2.132580  
                   2024-05-29 17:00:00+00:00    334.0         77.0  1.981707  
                   2024-05-29 18:00:00+00:00    407.0         78.0  1.843415  
                   2024-05-29 19:00:00+00:00   1463.0        202.0  1.915530  
   

Since the options contract are typically trade on 100 underline stocks we will trade multiplies of the 100 in our strategy

In [ ]:
option_prices['st_price'] = st_price
option_prices['und_price'] = ohlc_data.close
option_prices['dtexp'] = (pd.Timestamp(exp_date,tz='US/Pacific') - option_prices.index).days
option_prices#[option_prices.isna()]

,open,high,low,close,volume,trade_count,vwap,st_price,und_price,dtexp
timestamp,,,,,,,,,,
2024-01-18 05:00:00+00:00,3.87,5.75,3.87,5.50,185.0,57.0,5.130162,250.0,211.6100,155
2024-01-19 05:00:00+00:00,5.10,6.05,4.95,6.05,67.0,30.0,5.339254,250.0,215.0200,154
2024-01-22 05:00:00+00:00,5.42,5.49,5.20,5.30,22.0,9.0,5.430909,250.0,214.9300,151
2024-01-23 05:00:00+00:00,5.80,5.95,5.38,5.38,189.0,15.0,5.594709,250.0,211.5000,150
2024-01-24 05:00:00+00:00,5.25,6.49,5.25,5.60,89.0,38.0,5.816742,250.0,214.1300,149
...,...,...,...,...,...,...,...,...,...,...
2024-05-22 04:00:00+00:00,0.04,0.06,0.04,0.05,120.0,9.0,0.050583,250.0,186.2800,30
2024-05-23 04:00:00+00:00,0.04,0.07,0.04,0.05,247.0,12.0,0.050000,250.0,172.2100,29
2024-05-24 04:00:00+00:00,0.05,0.05,0.05,0.05,507.0,3.0,0.050000,250.0,174.5200,28


#### Greeks

In [ ]:

und_price = ohlc_data.close[0]
strike_price = st_price
int_rate = 0.04875
dtexp = (date(2024,4,15)-date(2024,1,1)).days
vola = hv
n_opt = 10
n_und_stocks = n_opt * 100
c = mibian.BS([und_price, strike_price, int_rate, dtexp], vola)
c.callDelta

0.549781939608853

In [ ]:
option_prices['delta'] = option_prices.apply(lambda x: mibian.BS([x.und_price, x.st_price, int_rate, x.dtexp], vola).callDelta,axis=1)*n_und_stocks
option_prices['gamma'] = option_prices.apply(lambda x: mibian.BS([x.und_price, x.st_price, int_rate, x.dtexp], vola).gamma,axis=1)*n_und_stocks
option_prices['theta'] = option_prices.apply(lambda x: mibian.BS([x.und_price, x.st_price, int_rate, x.dtexp], vola).callTheta,axis=1)*n_und_stocks
option_prices['vega'] = option_prices.apply(lambda x: mibian.BS([x.und_price, x.st_price, int_rate, x.dtexp], vola).vega,axis=1)*n_und_stocks
option_prices['IV'] = option_prices.apply(lambda x: mibian.BS([x.und_price, x.st_price, int_rate, x.dtexp], callPrice=x.close).impliedVolatility,axis=1)
option_prices.round(4)

,open,high,low,close,volume,trade_count,vwap,st_price,und_price,dtexp,delta,gamma,theta,vega,IV
timestamp,,,,,,,,,,,,,,,
2024-01-18 05:00:00+00:00,3.87,5.75,3.87,5.50,185.0,57.0,5.1302,250.0,211.6100,155,226.9024,7.2468,-40.4793,415.5445,31.4941
2024-01-19 05:00:00+00:00,5.10,6.05,4.95,6.05,67.0,30.0,5.3393,250.0,215.0200,154,251.2557,7.5652,-43.6334,445.0075,30.8228
2024-01-22 05:00:00+00:00,5.42,5.49,5.20,5.30,22.0,9.0,5.4309,250.0,214.9300,151,247.8480,7.5880,-43.7269,437.2830,29.5410
2024-01-23 05:00:00+00:00,5.80,5.95,5.38,5.38,189.0,15.0,5.5947,250.0,211.5000,150,221.4172,7.2687,-40.5580,402.9357,31.6772
2024-01-24 05:00:00+00:00,5.25,6.49,5.25,5.60,89.0,38.0,5.8167,250.0,214.1300,149,239.9397,7.5342,-43.0935,425.2536,30.7617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-22 04:00:00+00:00,0.04,0.06,0.04,0.05,120.0,9.0,0.0506,250.0,186.2800,30,0.3904,0.0877,-0.3794,0.7547,41.0156
2024-05-23 04:00:00+00:00,0.04,0.07,0.04,0.05,247.0,12.0,0.0500,250.0,172.2100,29,0.0070,0.0022,-0.0081,0.0156,51.7578
2024-05-24 04:00:00+00:00,0.05,0.05,0.05,0.05,507.0,3.0,0.0500,250.0,174.5200,28,0.0102,0.0031,-0.0118,0.0220,50.7812


### Delta Hedge

Short Calls and Long Delta Stocks

In [ ]:
interest = 1.05**(1/252)-1 #interest for borrowing the stocks
fee = 0.001 #transaction fees
backtest = option_prices[['close','und_price','delta']].copy().round(4)
backtest['capital'] = backtest.close[0]*n_und_stocks

backtest['n_stock'] = -backtest['delta'].astype(int) #number of stocks owed to broker
backtest['delta'] = backtest['delta'] -backtest['n_stock']
backtest['stock_owed'] = backtest['n_stock'] * backtest['und_price'] # amount of stock owed
backtest['int_on_stock'] = (backtest['stock_owed']*interest).abs() # interest on stock shuld be payed
backtest['transactions'] = backtest['stock_owed'].diff().fillna(0) #transaction amount
backtest['trading_fees'] =  (backtest['transactions'] * fee).abs().cumsum()
backtest['cumm_int_stock'] = backtest['int_on_stock'].cumsum()
backtest['capital'] = backtest['capital']-backtest['trading_fees'] -backtest['cumm_int_stock']
backtest


,close,und_price,delta,capital,n_stock,stock_owed,int_on_stock,transactions,trading_fees,cumm_int_stock
timestamp,,,,,,,,,,
2024-01-18 05:00:00+00:00,5.50,211.6100,452.9024,5490.739842,-226,-47823.86,9.260158,0.00,0.00000,9.260158
2024-01-19 05:00:00+00:00,6.05,215.0200,502.2557,5474.143439,-251,-53970.02,10.450242,-6146.16,6.14616,19.710401
2024-01-22 05:00:00+00:00,5.30,214.9300,494.8480,5462.981729,-247,-53087.71,10.279400,882.31,7.02847,29.989801
2024-01-23 05:00:00+00:00,5.38,211.5000,442.4172,5447.584939,-221,-46741.50,9.050580,6346.21,13.37468,39.040381
2024-01-24 05:00:00+00:00,5.60,214.1300,478.9397,5433.239927,-239,-51177.07,9.909442,-4435.57,17.81025,48.949823
...,...,...,...,...,...,...,...,...,...,...
2024-05-22 04:00:00+00:00,0.05,186.2800,0.3904,5081.658732,0,0.00,0.000000,0.00,172.49510,245.846168
2024-05-23 04:00:00+00:00,0.05,172.2100,0.0070,5081.658732,0,0.00,0.000000,0.00,172.49510,245.846168
2024-05-24 04:00:00+00:00,0.05,174.5200,0.0102,5081.658732,0,0.00,0.000000,0.00,172.49510,245.846168


In [ ]:
profit = backtest['capital'].iloc[-1]
profit

5081.6587324630855